In [3]:
#part 1
import pandas as pd
from flask import Flask, render_template, jsonify
import matplotlib.pyplot as plt
import seaborn as sns

# Load the main dataset
main_dataset_url = "malicious_phish.csv"  # Replace with your file path or URL
main_data = pd.read_csv(main_dataset_url)

# Create a DataFrame for the helper data (second file with URLs)
helper_data = pd.read_csv("spam_dataset.csv", header=None, names=["url"])
helper_data["type"] = "spam"  # Add a 'type' column with value 'spam'

# Merge the datasets based on a common key or concatenate them
merged_data = pd.concat([main_data, helper_data], ignore_index=True)

# Display the merged dataset
print(merged_data.head())

# Save the merged dataset to a new CSV file
merged_data.to_csv("merged_dataset_with_spam.csv", index=False)


ModuleNotFoundError: No module named 'pandas'

In [2]:
#part 2
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load the merged dataset
merged_dataset = pd.read_csv("merged_dataset_with_spam.csv")  # Replace with your file path

# Handle missing values (if any)
merged_dataset.dropna(inplace=True)  # For demonstration, dropping rows with any NaN values

# Data cleaning: Removing duplicates
merged_dataset.drop_duplicates(inplace=True)
# Encoding categorical variables
# Assuming 'type' column is categorical
encoder = OneHotEncoder(sparse=False)
encoded_types = pd.get_dummies(merged_dataset['type'], prefix='type')
merged_dataset = pd.concat([merged_dataset, encoded_types], axis=1)
merged_dataset.drop('type', axis=1, inplace=True)  # Drop the original 'type' column
# Save the preprocessed dataset to a new CSV file
merged_dataset.to_csv("preprocessed_merged_dataset.csv", index=False)


In [3]:


#part 3
import pandas as pd
from sklearn.utils import resample

# Load your dataset
dataset = pd.read_csv("preprocessed_merged_dataset.csv")

# Separate the data into benign and malicious classes
benign_data = dataset[dataset['type_benign'] == True]
malicious_data = dataset[dataset['type_benign'] == False]

# Find the class with the highest count (usually benign) for reference
max_class_count = benign_data.shape[0]

# Resample each malicious class to match the count of the benign class
resampled_data = []
for column in ['type_defacement', 'type_malware', 'type_phishing', 'type_spam']:
    malicious_class = malicious_data[malicious_data[column] == True]
    resampled_class = resample(malicious_class, n_samples=max_class_count, random_state=42)
    resampled_data.append(resampled_class)

# Combine the resampled malicious classes with the original benign class
balanced_dataset = pd.concat([benign_data] + resampled_data)

# Shuffle the dataset to ensure randomness
balanced_dataset = balanced_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the balanced dataset to a new CSV file
balanced_dataset.to_csv("balanced_dataset.csv", index=False)


In [ ]:

#part4
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load your balanced dataset
dataset = pd.read_csv("balanced_dataset.csv")  # Replace with your dataset filename

# Descriptive statistics
print("Descriptive Statistics:")
print(dataset.describe())

# Data distributions for each feature
print("\nData Distributions:")
for column in dataset.columns:
    if dataset[column].dtype == 'bool':
        # Plot counts for boolean columns
        sns.countplot(x=column, data=dataset)
    else:
        # Plot histograms for numeric columns
        sns.histplot(data=dataset, x=column, bins=20, kde=True)
    plt.title(f"Distribution of {column}")
    plt.show()

# Explore relationships between features and target variable
print("\nRelationships with Target Variable (type_benign):")
for column in dataset.columns:
    if column != 'type_benign':
        if dataset[column].dtype == 'bool':
            # Plot bar plots for boolean features
            sns.barplot(x='type_benign', y=column, data=dataset, ci=None)
        else:
            # Plot box plots for numeric features
            sns.boxplot(x='type_benign', y=column, data=dataset)
        plt.title(f"Relationship between {column} and type_benign")
        plt.show()


In [ ]:

#part 5
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load your balanced dataset
dataset = pd.read_csv("balanced_dataset.csv")  # Replace with your dataset filename

# Plot 1: Distribution of Malicious Types
plt.figure(figsize=(10, 6))
sns.countplot(x="type_benign", data=dataset)
plt.title("Distribution of Malicious Types")
plt.xlabel("Malicious Type")
plt.ylabel("Count")
plt.savefig("plot1.png")
plt.show()

# Plot 2: Correlation Heatmap
correlation_matrix = dataset.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.savefig("plot2.png")
plt.show()

# Extract URL length from the "url" column
dataset['url_length'] = dataset['url'].str.len()

# Plot 3: Box Plot of URL Length by Type
plt.figure(figsize=(10, 6))
sns.boxplot(x="type_benign", y="url_length", data=dataset)
plt.title("URL Length by Malicious Type")
plt.xlabel("Malicious Type")
plt.ylabel("URL Length")
plt.savefig("plot3.png")
plt.show()

# Plot 4: Pairplot for Selected Features
selected_features = ["type_malware", "type_phishing", "type_spam"]
# Create a pairplot
sns.pairplot(dataset[selected_features], hue="type_malware")
plt.suptitle("Pairplot of Selected Features by Malicious Type")
plt.savefig("plot4.png")
plt.show()


# Plot 5: Stacked Bar Plot of Categorical Features
# Modify the list of categorical features based on your dataset columns
categorical_features = ["type_defacement", "type_malware", "type_phishing", "type_spam"]

# Create a new DataFrame for the categorical features
categorical_data = dataset[categorical_features + ["type_benign"]]

# Melt the DataFrame to prepare for the stacked bar plot
stacked_data = categorical_data.melt(id_vars="type_benign", var_name="Categorical Feature")

# Plot 5: Stacked Bar Plot of Categorical Features
plt.figure(figsize=(12, 6))
sns.countplot(x="Categorical Feature", hue="value", data=stacked_data, hue_order=[True, False])
plt.title("Stacked Bar Plot of Categorical Features by Malicious Type")
plt.xlabel("Categorical Feature")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.savefig("plot5.png")
plt.show()



In [ ]:

#part 6
import pandas as pd
import re

# Load your balanced dataset
dataset = pd.read_csv("balanced_dataset.csv")  # Replace with your dataset filename

# Feature extraction function
def extract_features(url):
    features = {}

    # Feature 1: URL Length
    features['url_length'] = len(url)

    # Feature 2: Count of Special Characters
    special_characters = re.compile(r"[!@#$%^&*(),.?\":{}|<>]")
    features['special_char_count'] = len(re.findall(special_characters, url))

    # Feature 3: Count of Digits
    features['digit_count'] = len(re.findall(r"\d", url))

    # Feature 4: Count of Hyphens
    features['hyphen_count'] = len(re.findall(r"-", url))

    # Feature 5: Count of Dots
    features['dot_count'] = len(re.findall(r"\.", url))

    return features

# Apply feature extraction to the dataset
dataset['url_features'] = dataset['url'].apply(extract_features)

# Convert extracted features into separate columns
feature_columns = pd.DataFrame(dataset['url_features'].tolist())
dataset = pd.concat([dataset, feature_columns], axis=1)

# Drop the original 'url_features' column
dataset.drop('url_features', axis=1, inplace=True)

# Save the dataset with extracted features to a new CSV file
dataset.to_csv("dataset_with_extracted_features.csv", index=False)



In [ ]:

#part 7
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load your balanced dataset
dataset = pd.read_csv("dataset_with_extracted_features.csv")  # Replace with your dataset filename

# Define a list of columns to exclude from standardization
columns_to_exclude = ["url"]  # Add any other non-numeric or non-boolean columns

# Split the data into features (X) and target (y)
X = dataset.drop(['type_benign'], axis=1) if 'type_benign' in dataset.columns else dataset  # Check if 'type_benign' exists

# Remove columns to exclude from X
X = X.drop(columns_to_exclude, axis=1)
y = dataset['type_benign']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (scaling)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model 1: Logistic Regression
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(X_train, y_train)
y_pred_logistic = logistic_model.predict(X_test)
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print("Logistic Regression Accuracy:", accuracy_logistic)

# Model 2: Random Forest
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)
y_pred_rf = random_forest_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)


In [ ]:



#part 8
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

# Load your dataset with extracted features
dataset = pd.read_csv("dataset_with_extracted_features.csv")  # Replace with your dataset filename

# Exclude non-numeric columns (e.g., 'url') from scaling
numeric_columns = dataset.select_dtypes(include=['number']).columns
X = dataset[numeric_columns]
y = dataset['type_benign']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (scaling)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model 1: Logistic Regression
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(X_train, y_train)

# Model 2: Random Forest
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# Function to plot confusion matrix
def plot_confusion_matrix(model, X, y):
    y_pred = model.predict(X)
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    classes = ['Malicious', 'Benign']
    tick_marks = range(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

# Visualize Confusion Matrices
plot_confusion_matrix(logistic_model, X_test, y_test)
plot_confusion_matrix(random_forest_model, X_test, y_test)


In [ ]:
app = Flask(__name__)
@app.route('/')
def visualize_data():
    # Load your balanced dataset
    dataset = pd.read_csv("balanced_dataset.csv")

    # Plot 1: Distribution of Malicious Types
    plt.figure(figsize=(10, 6))
    sns.countplot(x="type_benign", data=dataset)
    plt.title("Distribution of Malicious Types")
    plt.xlabel("Malicious Type")
    plt.ylabel("Count")
    plt.savefig("plot1.png")

    # Plot 2: Correlation Heatmap
    correlation_matrix = dataset.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Correlation Heatmap")
    plt.savefig("plot2.png")

    # Extract URL length from the "url" column
    dataset['url_length'] = dataset['url'].str.len()

    # Plot 3: Box Plot of URL Length by Type
    plt.figure(figsize=(10, 6))
    sns.boxplot(x="type_benign", y="url_length", data=dataset)
    plt.title("URL Length by Malicious Type")
    plt.xlabel("Malicious Type")
    plt.ylabel("URL Length")
    plt.savefig("plot3.png")

    # Plot 4: Pairplot for Selected Features
    selected_features = ["type_malware", "type_phishing", "type_spam"]
    sns.pairplot(dataset[selected_features], hue="type_malware")
    plt.suptitle("Pairplot of Selected Features by Malicious Type")
    plt.savefig("plot4.png")

    # Plot 5: Stacked Bar Plot of Categorical Features
    categorical_features = ["type_defacement", "type_malware", "type_phishing", "type_spam"]
    categorical_data = dataset[categorical_features + ["type_benign"]]
    stacked_data = categorical_data.melt(id_vars="type_benign", var_name="Categorical Feature")
    plt.figure(figsize=(12, 6))
    sns.countplot(x="Categorical Feature", hue="value", data=stacked_data, hue_order=[True, False])
    plt.title("Stacked Bar Plot of Categorical Features by Malicious Type")
    plt.xlabel("Categorical Feature")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.savefig("plot5.png")

    # Return the HTML template with the rendered graphs
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)